In [1]:
!pwd

/opt/share/MERIDA


In [2]:
cd Code/Pytorch-UNet/

/opt/share/MERIDA/Code/Pytorch-UNet


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 39.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 KB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 KB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 KB 19.7 MB/s eta 0:00:00
  Created wheel for pathtools: filen

{"username":"ivangarcia07","key":"11b44cc47aa77c67bd8b1b9dd4f62e98"}

#MODIFICACIÓN EN BASE A NUESTROS REQUISITOS


Ahora mismo el modelo está aceptando como entrada una imagen y está produciendo como salida otra imagen con la respectiva máscara. Lo que yo tengo que realizar es una modificación del modelo para que en lugar de leer una sola imagen, lea una matriz de tamaño fijo, por ejemplo 100 x 100 con 8 dimensiones tal y como habíamos hablado (introduciendo por ello el path_loss, los respectivos snr de cada estación base que son 3, el despliegue de las 3 estaciones, así como el despliegue de los usuario). Como salida lo que tiene que dar es una matriz con el mismo tamaño que una de las dimensiones de la matriz de entrada en la cual se indica el ancho o cobertura que consume cada estación.

In [5]:
!ls /opt/share/MERIDA/Code/Test_Data/

0_femto_deploy	0_micro_deploy	0_outout.txt   0_pico_snr	     pathloss
0_femto_snr	0_micro_snr	0_pico_deploy  matriz_combinada.npy  users


In [33]:
!pip install natsort

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [68]:
import numpy as np
import os
from natsort import natsorted

lista = os.listdir('/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/deployment/')
lista = natsorted(lista)
print(lista)

for i in range(0,len(lista)-3):
    deploy_femto = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/deployment/'+str(i)+'_femto'
    deploy_micro = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/deployment/'+str(i)+'_micro'
    deploy_pico = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/deployment/'+str(i)+'_pico'
    if os.path.isfile(deploy_femto) and os.path.isfile(deploy_micro) and os.path.isfile(deploy_pico):
        snr_femto = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/snr/'+str(i)+'_femto'
        snr_micro = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/snr/'+str(i)+'_micro'
        snr_pico = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/snr/'+str(i)+'_pico'
        if os.path.isfile(snr_femto) and os.path.isfile(snr_micro) and os.path.isfile(snr_pico):
            user_data = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/users'
            pathloss_data = '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/pathloss'
            
            matriz_femto_snr = np.loadtxt(snr_femto, dtype=float)
            matriz_micro_snr = np.loadtxt(snr_micro, dtype=float)
            matriz_pico_snr = np.loadtxt(snr_pico, dtype=float)
            matriz_femto_deploy = np.loadtxt(deploy_femto, dtype=float)
            matriz_micro_deploy = np.loadtxt(deploy_micro, dtype=float)
            matriz_pico_deploy = np.loadtxt(deploy_pico, dtype=float)
            matriz_pathloss = np.loadtxt(pathloss_data, dtype=float)
            matriz_users = np.loadtxt(user_data, dtype=float)

            # Combina las matrices en una sola matriz de 8 dimensiones
            matriz_combinada = np.array([matriz_femto_snr, matriz_micro_snr, matriz_pico_snr,
                                         matriz_femto_deploy, matriz_micro_deploy, matriz_pico_deploy,
                                         matriz_pathloss, matriz_users])
            
            ruta_guardado_binario = '/opt/share/MERIDA/DATASET-TEST/IN/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matriz_combinada)
            
            # Leer los datos desde el archivo CSV
            data = np.genfromtxt('/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/usersInfo/'+str(i)+'.csv', delimiter=',', skip_header=1)
            # Obtener las coordenadas x, y y capacidad
            x = data[:, 1].astype(int)
            y = data[:, 2].astype(int)
            capacity = data[:, 4]
            # Obtener las dimensiones de la matriz
            max_x, max_y = np.max(x), np.max(y)
            print(max_x, max_y)
            # Crear una matriz de ceros con las dimensiones máximas
            matrix = np.zeros((max_x + 1, max_y + 1))
            # Asignar los valores de capacidad en las posiciones correspondientes
            matrix[x, y] = capacity
            # Guardar la matriz en formato binario de NumPy (npy)
            ruta_guardado_binario = '/opt/share/MERIDA/DATASET-TEST/OUT/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matrix)
            
#print(matriz_combinada.shape)
#print(matriz_combinada[1].shape)

# Verifica la forma de la matriz combinada
#print("Forma de la matriz combinada:", matriz_combinada.shape)

# Ruta donde guardar el archivo combinado en formato binario de NumPy
#ruta_guardado_binario = '/content/drive/MyDrive/MERIDA/Test_Data/matriz_combinada_v2.npy'
#ruta_guardado_binario = '/content/matriz_combinada.npy'

# Guarda la matriz combinada en un archivo binario de NumPy
#np.save(ruta_guardado_binario, matriz_combinada)

#print(f"Matriz combinada guardada en formato binario de NumPy en: {ruta_guardado_binario}")

['0_femto', '0_micro', '0_pico', '1_femto', '1_micro', '1_pico', '2_femto', '2_micro', '2_pico', '3_femto', '3_micro', '3_pico', '4_femto', '4_micro', '4_pico', '5_femto', '5_micro', '5_pico', '6_femto', '6_micro', '6_pico', '7_femto', '7_micro', '7_pico', '8_femto', '8_micro', '8_pico', '9_femto', '9_micro', '9_pico', '10_femto', '10_micro', '10_pico', '11_femto', '11_micro', '11_pico', '12_femto', '12_micro', '12_pico', '13_femto', '13_micro', '13_pico', '14_femto', '14_micro', '14_pico', '15_femto', '15_micro', '15_pico', '16_femto', '16_micro', '16_pico', '17_femto', '17_micro', '17_pico', '18_femto', '18_micro', '18_pico', '19_femto', '19_micro', '19_pico', '20_femto', '20_micro', '20_pico', '21_femto', '21_micro', '21_pico', '22_femto', '22_micro', '22_pico', '23_femto', '23_micro', '23_pico', '24_femto', '24_micro', '24_pico', '25_femto', '25_micro', '25_pico', '26_femto', '26_micro', '26_pico', '27_femto', '27_micro', '27_pico', '28_femto', '28_micro', '28_pico', '29_femto', '2

99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 9

In [70]:
import numpy as np
import os
from natsort import natsorted

lista = os.listdir('/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/deployment/')
lista = natsorted(lista)
print(lista)

for i in range(0,len(lista)-3):
    deploy_femto = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/deployment/'+str(i)+'_femto'
    deploy_micro = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/deployment/'+str(i)+'_micro'
    deploy_pico = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/deployment/'+str(i)+'_pico'
    if os.path.isfile(deploy_femto) and os.path.isfile(deploy_micro) and os.path.isfile(deploy_pico):
        snr_femto = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/snr/'+str(i)+'_femto'
        snr_micro = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/snr/'+str(i)+'_micro'
        snr_pico = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/snr/'+str(i)+'_pico'
        if os.path.isfile(snr_femto) and os.path.isfile(snr_micro) and os.path.isfile(snr_pico):
            user_data = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/users'
            pathloss_data = '/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/pathloss'
            
            matriz_femto_snr = np.loadtxt(snr_femto, dtype=float)
            matriz_micro_snr = np.loadtxt(snr_micro, dtype=float)
            matriz_pico_snr = np.loadtxt(snr_pico, dtype=float)
            matriz_femto_deploy = np.loadtxt(deploy_femto, dtype=float)
            matriz_micro_deploy = np.loadtxt(deploy_micro, dtype=float)
            matriz_pico_deploy = np.loadtxt(deploy_pico, dtype=float)
            matriz_pathloss = np.loadtxt(pathloss_data, dtype=float)
            matriz_users = np.loadtxt(user_data, dtype=float)

            # Combina las matrices en una sola matriz de 8 dimensiones
            matriz_combinada = np.array([matriz_femto_snr, matriz_micro_snr, matriz_pico_snr,
                                         matriz_femto_deploy, matriz_micro_deploy, matriz_pico_deploy,
                                         matriz_pathloss, matriz_users])
            
            ruta_guardado_binario = '/opt/share/MERIDA/DATASET-TEST2/IN/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matriz_combinada)
            
            # Leer los datos desde el archivo CSV
            data = np.genfromtxt('/opt/share/MERIDA/Data/adaptive_1/1-0-4605879/ia/raw/usersInfo/'+str(i)+'.csv', delimiter=',', skip_header=1)
            # Obtener las coordenadas x, y y capacidad
            x = data[:, 1].astype(int)
            y = data[:, 2].astype(int)
            capacity = data[:, 4]
            # Obtener las dimensiones de la matriz
            max_x, max_y = np.max(x), np.max(y)
            print(max_x, max_y)
            # Crear una matriz de ceros con las dimensiones máximas
            matrix = np.zeros((max_x + 1, max_y + 1))
            # Asignar los valores de capacidad en las posiciones correspondientes
            matrix[x, y] = capacity
            # Guardar la matriz en formato binario de NumPy (npy)
            ruta_guardado_binario = '/opt/share/MERIDA/DATASET-TEST2/OUT/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matrix)
            
#print(matriz_combinada.shape)
#print(matriz_combinada[1].shape)

# Verifica la forma de la matriz combinada
#print("Forma de la matriz combinada:", matriz_combinada.shape)

# Ruta donde guardar el archivo combinado en formato binario de NumPy
#ruta_guardado_binario = '/content/drive/MyDrive/MERIDA/Test_Data/matriz_combinada_v2.npy'
#ruta_guardado_binario = '/content/matriz_combinada.npy'

# Guarda la matriz combinada en un archivo binario de NumPy
#np.save(ruta_guardado_binario, matriz_combinada)

#print(f"Matriz combinada guardada en formato binario de NumPy en: {ruta_guardado_binario}")

['0_femto', '0_micro', '0_pico', '1_femto', '1_micro', '1_pico', '2_femto', '2_micro', '2_pico', '3_femto', '3_micro', '3_pico', '4_femto', '4_micro', '4_pico', '5_femto', '5_micro', '5_pico', '6_femto', '6_micro', '6_pico', '7_femto', '7_micro', '7_pico', '8_femto', '8_micro', '8_pico', '9_femto', '9_micro', '9_pico', '10_femto', '10_micro', '10_pico', '11_femto', '11_micro', '11_pico', '12_femto', '12_micro', '12_pico', '13_femto', '13_micro', '13_pico', '14_femto', '14_micro', '14_pico', '15_femto', '15_micro', '15_pico', '16_femto', '16_micro', '16_pico', '17_femto', '17_micro', '17_pico', '18_femto', '18_micro', '18_pico', '19_femto', '19_micro', '19_pico', '20_femto', '20_micro', '20_pico', '21_femto', '21_micro', '21_pico', '22_femto', '22_micro', '22_pico', '23_femto', '23_micro', '23_pico', '24_femto', '24_micro', '24_pico', '25_femto', '25_micro', '25_pico', '26_femto', '26_micro', '26_pico', '27_femto', '27_micro', '27_pico', '28_femto', '28_micro', '28_pico', '29_femto', '2

99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 99
99 9

In [42]:
# Ruta del archivo binario de NumPy que contiene la matriz combinada
ruta_archivo_binario = '/opt/share/MERIDA/DATASET-TEST/IN/matriz_combinada461.npy'

# Carga la matriz combinada desde el archivo binario
matriz_combinada_leida = np.load(ruta_archivo_binario)

# Ahora, puedes usar la matriz_combinada_leida en tu código
print("Matriz combinada cargada exitosamente:")
#print(matriz_combinada_leida)

print("Forma de la matriz combinada:")
print(matriz_combinada_leida.shape)

print(matriz_combinada_leida[7])

Matriz combinada cargada exitosamente:
Forma de la matriz combinada:
(8, 100, 100)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
import numpy as np

matriz_output = np.loadtxt('/content/drive/MyDrive/MERIDA/Test_Data/0_outout.txt', dtype=float)

# Verifica la forma de la matriz combinada
print("Forma de la matriz combinada:", matriz_output.shape)

for i in range(matriz_output.shape[-1]):
    tipo_de_datos = matriz_output[..., i].dtype
    #print(f"Tipo de datos de la dimensión {i + 1}: {tipo_de_datos}")


# Ruta donde guardar el archivo combinado en formato binario de NumPy
#ruta_guardado_binario = '/content/drive/MyDrive/MERIDA/examples-output/matriz_combinada_output.npy'
ruta_guardado_binario = '/content/drive/MyDrive/MERIDA/examples-output/4.npy'

# Guarda la matriz combinada en un archivo binario de NumPy
#np.save(ruta_guardado_binario, matriz_output)

print(f"Matriz combinada guardada en formato binario de NumPy en: {ruta_guardado_binario}")


Forma de la matriz combinada: (480, 640)
Matriz combinada guardada en formato binario de NumPy en: /content/drive/MyDrive/MERIDA/examples-output/4.npy


# Primer paso:

## Modificación del utils. (/content/drive/MyDrive/MERIDA/Pytorch-UNet/utils/data_loading.py) con el objetivo de crear un nuevo class Dataset con el siguiente contenido:

In [8]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 70.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 37.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 71.7 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 M

In [12]:
import logging
import numpy as np
import torch
from PIL import Image
from functools import lru_cache
from functools import partial
from itertools import repeat
from multiprocessing import Pool
from os import listdir
from os.path import splitext, isfile, join
from pathlib import Path
from torch.utils.data import Dataset
from tqdm import tqdm


def load_npy(filename):
    return np.load(filename)


class CustomDataset2(Dataset):
    def __init__(self, data_dir: str, target_dir: str, scale: float = 1.0):
        self.data_dir = Path(data_dir)
        self.target_dir = Path(target_dir)
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'
        self.scale = scale

        # Assuming both data and target files have the same names
        self.ids = [splitext(file)[0] for file in listdir(data_dir) if isfile(join(data_dir, file)) and not file.startswith('.')]
        if not self.ids:
            raise RuntimeError(f'No input file found in {data_dir}, make sure you put your data files there')

        logging.info(f'Creating dataset with {len(self.ids)} examples')

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        name = self.ids[idx]
        data_file = self.data_dir / (name + '.npy')
        target_file = self.target_dir / (name + '.npy')

        print(self.data_dir / (name + '.npy'))
        print(self.target_dir / (name + '.npy'))

        data = load_npy(data_file)
        target = load_npy(target_file)

        # Assuming data and target have the same shape
        #assert data.shape == target.shape, f'Data and target shapes do not match for {name}: {data.shape} and {target.shape}'

        #ONLY TO RESIZE
        #data_resized, target_resized = self.preprocess(data, target, self.scale)

        return {
            'data': torch.as_tensor(data.copy()).float().contiguous(),
            'target': torch.as_tensor(target.copy()).float().contiguous()
        }

In [10]:
!pwd

/opt/share/MERIDA/Code/Pytorch-UNet


In [13]:
dataset = CustomDataset2(data_dir='/opt/share/MERIDA/Code/examples-input/', target_dir='/opt/share/MERIDA/Code/examples-output')
print(len(dataset))

sample = dataset[0]
#print(sample['data'])
print(dataset[0]['data'].shape)

100
/opt/share/MERIDA/Code/examples-input/51.npy
/opt/share/MERIDA/Code/examples-output/51.npy
/opt/share/MERIDA/Code/examples-input/51.npy
/opt/share/MERIDA/Code/examples-output/51.npy
torch.Size([100, 100, 8])


In [14]:
from torch.utils.data import DataLoader, random_split
import torch

# 2. Split into train / validation partitions
n_val = int(len(dataset) * 0.1)
n_train = len(dataset) - n_val
train_set, val_set = random_split(dataset, [n_train, n_val], generator=torch.Generator().manual_seed(0))


In [15]:
print(dataset)

In [16]:
import os

loader_args = dict(batch_size=1, num_workers=os.cpu_count(), pin_memory=True)
train_loader = DataLoader(train_set, shuffle=True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

In [17]:
for batch in train_loader:
  images, true_masks = batch['data'], batch['target']
  print(images.shape)

/opt/share/MERIDA/Code/examples-input/8.npy/opt/share/MERIDA/Code/examples-input/88.npy/opt/share/MERIDA/Code/examples-input/46.npy/opt/share/MERIDA/Code/examples-input/69.npy/opt/share/MERIDA/Code/examples-input/25.npy/opt/share/MERIDA/Code/examples-input/66.npy/opt/share/MERIDA/Code/examples-input/35.npy/opt/share/MERIDA/Code/examples-input/6.npy
/opt/share/MERIDA/Code/examples-input/26.npy/opt/share/MERIDA/Code/examples-input/48.npy
/opt/share/MERIDA/Code/examples-input/27.npy/opt/share/MERIDA/Code/examples-input/64.npy

/opt/share/MERIDA/Code/examples-output/88.npy

/opt/share/MERIDA/Code/examples-input/14.npy/opt/share/MERIDA/Code/examples-output/69.npy

/opt/share/MERIDA/Code/examples-output/46.npy


/opt/share/MERIDA/Code/examples-input/17.npy/opt/share/MERIDA/Code/examples-output/27.npy/opt/share/MERIDA/Code/examples-output/35.npy/opt/share/MERIDA/Code/examples-output/26.npy

/opt/share/MERIDA/Code/examples-output/6.npy
/opt/share/MERIDA/Code/examples-output/64.npy/opt/share/ME

In [18]:
import os

def remove_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Replace 'your_folder_path' with the path of the folder you want to clear
folder_to_clear = '/opt/share/MERIDA/Code/Pytorch-UNet/checkpoints/'
remove_files_in_folder(folder_to_clear)


In [ ]:
!python train_v5.py --amp --epochs 500

In [52]:
print('a')

a


In [21]:
import numpy as np

# Ruta al archivo .npy
archivo_npy = '/opt/share/MERIDA/out/matriz_combinada15.npy'
# Cargar el archivo .npy
matriz = np.load(archivo_npy)
# Obtener los índices de los elementos no cero
indices_no_cero = np.nonzero(matriz)
# Imprimir los valores no cero y sus índices
for i, j in zip(indices_no_cero[0], indices_no_cero[1]):
    valor = matriz[i, j]
    print(f"Valor no cero en la posición ({i}, {j}): {valor}")

Valor no cero en la posición (0, 14): 35.40391148221293
Valor no cero en la posición (0, 38): 12.432333746388633
Valor no cero en la posición (0, 92): 0.9337228233247069
Valor no cero en la posición (1, 12): 20.152267816109887
Valor no cero en la posición (1, 85): 12.612240652308879
Valor no cero en la posición (1, 90): 0.9409441693992718
Valor no cero en la posición (2, 15): 28.02165938143815
Valor no cero en la posición (2, 29): 33.88401588944908
Valor no cero en la posición (3, 78): 1.0164118354145362
Valor no cero en la posición (3, 88): 0.9504491206270869
Valor no cero en la posición (4, 8): 79.43336323963268
Valor no cero en la posición (4, 15): 82.34657860293107
Valor no cero en la posición (4, 27): 31.011885440492257
Valor no cero en la posición (5, 85): 34.48060885646471
Valor no cero en la posición (7, 17): 74.44059835872577
Valor no cero en la posición (7, 25): 48.46909208056047
Valor no cero en la posición (7, 46): 50.73519259576901
Valor no cero en la posición (7, 94): 0.8

In [92]:
!cp /opt/share/MERIDA/Code/Pytorch-UNet/checkpoints/checkpoint_epoch137.pth /opt/share/MERIDA/Code/Pytorch-UNet/MODEL.pth

In [ ]:
/home/ivangarcia/Desktop/DOCKTENSORFINAL/otro/models/share/MERIDA/DATASET-TEST/IN/matriz_combinada93.npy

In [93]:
!python predictv2.py -i '/opt/share/MERIDA/DATASET-TEST/PRUEBAS/IN/matriz_combinada11.npy' -o '/opt/share/MERIDA/SALIDA/salida11.npy'

INFO: Loading model MODEL.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting values for matrix /opt/share/MERIDA/DATASET-TEST/PRUEBAS/IN/matriz_combinada11.npy ...
(8, 100, 100)
INFO: Regression values saved to /opt/share/MERIDA/SALIDA/salida11.npy


In [56]:
import numpy as np

# Especifica la ruta al archivo .npy
file_path = '/opt/share/MERIDA/SALIDA/salida505.npy'

# Carga el archivo .npy
data = np.load(file_path)
print(data.shape)

# Ahora, 'data' contendrá los datos almacenados en el archivo .npy
print(repr(data))


(1, 1, 100, 100)
array([[[[16.364054 , 22.953623 , 21.772635 , ..., 16.033255 ,
          12.753892 ,  9.3704405],
         [15.769825 , 19.762627 , 12.752591 , ...,  1.0547212,
           1.5541915,  2.4702642],
         [10.527951 , 20.22148  ,  9.3701105, ...,  0.9144805,
           1.0633397,  4.0694656],
         ...,
         [11.225814 ,  7.9533916,  3.8422847, ...,  2.0961483,
          25.50405  , 11.9784155],
         [ 9.424966 , 14.376517 ,  7.3207994, ..., 20.84469  ,
           9.766868 , 15.342599 ],
         [18.385332 , 22.622322 , 16.567123 , ..., 28.293444 ,
          29.807238 , 26.367527 ]]]], dtype=float32)


In [98]:
import numpy as np

# Cargar la matriz desde el archivo .npy
matriz = np.load('/opt/share/MERIDA/DATASET-TEST/PRUEBAS/OUT/matriz_combinada11.npy')
matriz_pred = np.load('/opt/share/MERIDA/SALIDA/salida11.npy')

print('A',matriz_pred[0][0].shape, matriz.shape)

# Obtener las posiciones donde los elementos no son cero
posiciones_no_cero = np.nonzero(matriz)
posiciones_cero = np.where(matriz==0)

print('POSICIONES DISTINTAS DE CERO')
# Imprimir las posiciones y los valores correspondientes
for fila, columna in zip(*posiciones_no_cero):
    valor = matriz[fila, columna]
    valorpred = matriz_pred[0][0][fila,columna]
    print(f"Posición ({fila}, {columna}): Valor GT = {valor}, Valor PRED = {valorpred}")
    
    
print('POSICIONES DE CERO')
# Imprimir las posiciones y los valores correspondientes
for fila, columna in zip(*posiciones_cero):
    valor = matriz[fila, columna]
    valorpred = matriz_pred[0][0][fila,columna]
    print(f"Posición ({fila}, {columna}): Valor GT = {valor}, Valor PRED = {valorpred}")


A (100, 100) (100, 100)
POSICIONES DISTINTAS DE CERO
Posición (0, 14): Valor GT = 35.40391148221293, Valor PRED = 34.619869232177734
Posición (0, 38): Valor GT = 12.432333746388633, Valor PRED = 12.401328086853027
Posición (0, 92): Valor GT = 0.9337228233247069, Valor PRED = 11.88453483581543
Posición (1, 12): Valor GT = 20.152267816109887, Valor PRED = 20.294986724853516
Posición (1, 85): Valor GT = 12.612240652308879, Valor PRED = 11.72610092163086
Posición (1, 90): Valor GT = 0.9409441693992718, Valor PRED = 9.615498542785645
Posición (2, 15): Valor GT = 28.02165938143815, Valor PRED = 27.82059669494629
Posición (2, 29): Valor GT = 33.88401588944908, Valor PRED = 32.902767181396484
Posición (3, 78): Valor GT = 1.0164118354145362, Valor PRED = 1.0005711317062378
Posición (3, 88): Valor GT = 0.9504491206270869, Valor PRED = 25.442052841186523
Posición (4, 8): Valor GT = 79.43336323963268, Valor PRED = 78.16117095947266
Posición (4, 15): Valor GT = 82.34657860293107, Valor PRED = 80.61

Posición (27, 19): Valor GT = 0.0, Valor PRED = 30.518163681030273
Posición (27, 20): Valor GT = 0.0, Valor PRED = 7.172779560089111
Posición (27, 21): Valor GT = 0.0, Valor PRED = 26.217090606689453
Posición (27, 23): Valor GT = 0.0, Valor PRED = 17.37706184387207
Posición (27, 24): Valor GT = 0.0, Valor PRED = 18.841331481933594
Posición (27, 25): Valor GT = 0.0, Valor PRED = 22.986936569213867
Posición (27, 26): Valor GT = 0.0, Valor PRED = 20.46485710144043
Posición (27, 27): Valor GT = 0.0, Valor PRED = 34.61540985107422
Posición (27, 28): Valor GT = 0.0, Valor PRED = 26.169729232788086
Posición (27, 29): Valor GT = 0.0, Valor PRED = 24.652921676635742
Posición (27, 30): Valor GT = 0.0, Valor PRED = 25.713247299194336
Posición (27, 31): Valor GT = 0.0, Valor PRED = 29.848072052001953
Posición (27, 32): Valor GT = 0.0, Valor PRED = 26.863670349121094
Posición (27, 33): Valor GT = 0.0, Valor PRED = 17.80803871154785
Posición (27, 34): Valor GT = 0.0, Valor PRED = 9.723383903503418
P

Posición (61, 0): Valor GT = 0.0, Valor PRED = 20.627098083496094
Posición (61, 1): Valor GT = 0.0, Valor PRED = 20.689987182617188
Posición (61, 2): Valor GT = 0.0, Valor PRED = 21.094066619873047
Posición (61, 3): Valor GT = 0.0, Valor PRED = 14.338754653930664
Posición (61, 4): Valor GT = 0.0, Valor PRED = 25.26174545288086
Posición (61, 5): Valor GT = 0.0, Valor PRED = 31.17630958557129
Posición (61, 6): Valor GT = 0.0, Valor PRED = 23.576885223388672
Posición (61, 7): Valor GT = 0.0, Valor PRED = 16.572500228881836
Posición (61, 8): Valor GT = 0.0, Valor PRED = 34.3354377746582
Posición (61, 9): Valor GT = 0.0, Valor PRED = 32.95746612548828
Posición (61, 10): Valor GT = 0.0, Valor PRED = 22.90687370300293
Posición (61, 11): Valor GT = 0.0, Valor PRED = 15.895325660705566
Posición (61, 12): Valor GT = 0.0, Valor PRED = 35.1693229675293
Posición (61, 13): Valor GT = 0.0, Valor PRED = 37.13676071166992
Posición (61, 14): Valor GT = 0.0, Valor PRED = 34.74401092529297
Posición (61, 1

Posición (96, 95): Valor GT = 0.0, Valor PRED = 17.426870346069336
Posición (96, 96): Valor GT = 0.0, Valor PRED = 19.622695922851562
Posición (96, 97): Valor GT = 0.0, Valor PRED = 35.18254852294922
Posición (96, 98): Valor GT = 0.0, Valor PRED = 46.492122650146484
Posición (96, 99): Valor GT = 0.0, Valor PRED = 13.835039138793945
Posición (97, 0): Valor GT = 0.0, Valor PRED = 13.792516708374023
Posición (97, 1): Valor GT = 0.0, Valor PRED = 22.387134552001953
Posición (97, 2): Valor GT = 0.0, Valor PRED = 6.111021041870117
Posición (97, 3): Valor GT = 0.0, Valor PRED = 9.271424293518066
Posición (97, 4): Valor GT = 0.0, Valor PRED = 20.485231399536133
Posición (97, 5): Valor GT = 0.0, Valor PRED = 15.20875072479248
Posición (97, 6): Valor GT = 0.0, Valor PRED = 5.791708946228027
Posición (97, 7): Valor GT = 0.0, Valor PRED = 10.141491889953613
Posición (97, 8): Valor GT = 0.0, Valor PRED = 27.36241340637207
Posición (97, 9): Valor GT = 0.0, Valor PRED = 16.17924690246582
Posición (97

In [67]:
print(matriz[0,0],matriz_pred[0][0][0,0])

0.0 16.364054


In [ ]:
import numpy as np

# Assume 'data' is your NumPy array
data = np.load(file_path)

# Specify the CSV file path
csv_file_path = '/content/output.csv'

# Save the NumPy array to a CSV file
np.savetxt(csv_file_path, data, delimiter=',')

print(f'The NumPy array has been saved to {csv_file_path}.')


The NumPy array has been saved to /content/output.csv.
